In [1]:
from dataReadFromDisk import *

In [3]:
df = load_df_from_csv()

DataFrame loaded from: E:\Notebooks\osu\processed-metadata\processed-metadata.csv


In [4]:
df = df[df['total_length'] < 600]
# df.iloc[7]
# df = df[df["audio"] == "0013ddfd8bd55fdccc0b253e313b7a60"]
df = df[:1]

In [7]:
df

,audio,split,folder,beatmapset_id,beatmap_id,approved,total_length,hit_length,version,file_md5,...,difficultyrating,StackLeniency,DistanceSpacing,BeatDivisor,HPDrainRate,CircleSize,OverallDifficulty,ApproachRate,SliderTickRate,SliderMultiplier
0,00010f0eb02ee131aacac54bf72d5444,train,5bea04ca275826768fa4f8411822c388,830266,1739583,1,221,215,Zero Vector,00010f0eb02ee131aacac54bf72d5444,...,4.93655,0.0,1.2,4,5.0,5.0,8.0,8.0,1,1.2


In [4]:
# testa,testb = process_audio_and_beatmap_for_model_single(df.iloc[0])
# testa.shape
# # for h in testb:
# #     print(h)

In [4]:
for index, row in df.iterrows():
    audio,beatmap,params = new_process_audio_and_beatmap_for_model_single(row)

In [5]:
beatmap.shape

(939, 3)

In [6]:
for b in beatmap:
    print(b)

[3194.    0.    5.]
[3408.    0.    5.]
[3622.    0.    1.]
[3837.    0.    5.]
[4051.    0.    1.]
[4265.    0.    5.]
[5122.    0.    1.]
[5979.    0.    5.]
[6622.    0.    1.]
[7265.    0.    5.]
[7694.    0.    1.]
[8337.    0.    1.]
[10051.     0.     5.]
[10265.     0.     5.]
[10479.     0.     1.]
[10694.     0.     5.]
[10908.     0.     1.]
[11122.     0.     5.]
[11979.     0.     1.]
[12836.     0.     5.]
[13479.     0.     1.]
[14122.     0.     5.]
[14551.     0.     1.]
[16265.     0.     1.]
[21408.     0.     5.]
[21515.     0.     1.]
[21622.     0.     1.]
[21837.     0.     5.]
[21944.     0.     1.]
[22051.     0.     1.]
[22265.     0.     5.]
[22372.     0.     1.]
[22479.     0.     1.]
[22694.     0.     5.]
[22801.     0.     1.]
[22908.     0.     1.]
[23122.     0.     5.]
[23229.     0.     1.]
[23337.     0.     1.]
[23551.     0.     5.]
[23658.     0.     1.]
[23765.     0.     1.]
[23979.     0.     5.]
[24194.     0.     1.]
[24622.     0.     1.]
[

In [8]:
dataset,num_of_batches,processed_audio_list,processed_beatmap_list,hyper_params_list = new_process_audio_and_beatmap_for_model_all_and_save(df)

In [9]:
num_of_batches

20

In [10]:
for a,b,p in dataset:
    print(p.shape)

(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)
(11,)


In [ ]:
# for h in processed_beatmap_list[19]:
#     print(h)

In [16]:

import tensorflow as tf
from tensorflow import keras

class PositionalEncoding(keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model
        )
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

def create_transformer_model(input_shape, output_shape, params_shape, num_layers=4, d_model=128, num_heads=8, dff=512, dropout_rate=0.1):
    # Input layers
    inputs = keras.Input(shape=input_shape)
    params_input = keras.Input(shape=params_shape)

    # Embedding layer for input
    embedding = keras.layers.Dense(d_model)(inputs)
    
    # Add positional encoding
    pos_encoding = PositionalEncoding(input_shape[0], d_model)
    x = pos_encoding(embedding)

    # Transformer layers
    for _ in range(num_layers):
        # Multi-head attention
        attention = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        attn_output = attention(x, x)
        attn_output = keras.layers.Dropout(dropout_rate)(attn_output)
        out1 = keras.layers.LayerNormalization(epsilon=1e-6)(x + attn_output)

        # Feed-forward network
        ffn = keras.Sequential([
            keras.layers.Dense(dff, activation='relu'),
            keras.layers.Dense(d_model)
        ])
        ffn_output = ffn(out1)
        ffn_output = keras.layers.Dropout(dropout_rate)(ffn_output)
        x = keras.layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

    # Concatenate with params input (broadcasting params across time steps)
    params_broadcasted = keras.layers.RepeatVector(input_shape[0])(params_input)
    x = keras.layers.Concatenate()([x, params_broadcasted])

    # Final dense layers
    x = keras.layers.Dense(256, activation='relu')(x)
    x = keras.layers.Dropout(dropout_rate)(x)
    outputs = keras.layers.Dense(output_shape[1])(x)

    # Create model
    model = keras.Model(inputs=[inputs, params_input], outputs=outputs)
    return model

# Create and compile the model
input_shape = (60000, 14)
output_shape = (60000, 4)
params_shape = (11,)

model = create_transformer_model(input_shape, output_shape, params_shape)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Print model summary
model.summary()

# Function to prepare the dataset for training
def prepare_dataset(dataset):
    return dataset.map(lambda x, y, z: ((x, z), y)).batch(32).prefetch(tf.data.AUTOTUNE)

# Create and prepare the dataset
train_dataset = prepare_dataset(dataset)

# Train the model
history = model.fit(dataset, epochs=10)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 60000, 14)]  0           []                               
                                                                                                  
 dense_44 (Dense)               (None, 60000, 128)   1920        ['input_9[0][0]']                
                                                                                                  
 positional_encoding_4 (Positio  (None, 60000, 128)  0           ['dense_44[0][0]']               
 nalEncoding)                                                                                     
                                                                                                  
 multi_head_attention_16 (Multi  (None, 60000, 128)  527488      ['positional_encoding_4[0][

ValueError: in user code:

    File "c:\Users\darsh\anaconda3\envs\latestEnv\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\darsh\anaconda3\envs\latestEnv\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\darsh\anaconda3\envs\latestEnv\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\darsh\anaconda3\envs\latestEnv\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\darsh\anaconda3\envs\latestEnv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\darsh\anaconda3\envs\latestEnv\lib\site-packages\keras\engine\input_spec.py", li

In [ ]:
# from makeBeatmap import make_beatmap_from_output
# make_beatmap_from_output(df.iloc[20],processed_beatmap_list[20])

In [ ]:
# for input_data, output_data in dataset.take(1):
#     print("First input:")
#     print(input_data.numpy()[0][0])
#     print("First output:")
#     print(output_data.numpy()[0][0])

In [ ]:
# import numpy as np
# import tensorflow as tf

# model = tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(None, 142)),  # Input layer for variable length sequences with 142 features
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256 , return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256 , return_sequences=True)),
#     # tf.keras.layers.Dense(64, activation='relu'),  # Optional dense layer
#     tf.keras.layers.Dense(35)  # Output layer, size 35 to match your output data dimension
# ])

# model.compile(optimizer='adam', loss=tf.keras.losses.Huber())

# # Training the model
# model.fit(dataset, epochs=10, steps_per_epoch=num_of_batches) # instead of 50 make it number of songs in final processed data

In [ ]:
# import tensorflow as tf

# # Check if GPU is available
# gpus = tf.config.experimental.list_physical_devices()
# gpus

In [ ]:
# print(tf.config.list_physical_devices('GPU'))

In [ ]:
# tf.__version__

In [ ]:
# import tensorflow as tf

# # # Ensure TensorFlow is set up to use the GPU
# # physical_devices = tf.config.list_physical_devices('GPU')
# # if len(physical_devices) > 0:
# #     tf.config.experimental.set_memory_growth(physical_devices[0], True)
# #     print("GPU is available and configured.")
# # else:
# #     print("GPU is not available.")

# # Check for GPU availability
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# def custom_loss(y_true, y_pred):
#     hit_weight = 5.0  # Higher weight for hit objects
#     no_hit_weight = 1.0  # Lower weight for no hit objects (zeros)
    
#     # Compute the Mean Squared Error (MSE) loss
#     loss = tf.keras.losses.mean_squared_error(y_true, y_pred)  # Shape: (batch_size, time_steps, output_dim)
    
#     # Create a weight matrix based on the true values (broadcast across batch and time steps)
#     weights = tf.where(tf.not_equal(y_true, 0), hit_weight, no_hit_weight)  # Shape: (batch_size, time_steps, output_dim)
    
#     # Ensure the weights have the same shape as the loss
#     weights = tf.cast(weights, tf.float32)
    
#     # Apply the weights to the loss (element-wise multiplication)
#     weighted_loss = tf.multiply(loss, weights)  # Element-wise multiplication
    
#     # Return the mean loss across the batch and time steps
#     return tf.reduce_mean(weighted_loss)

# # Define the model
# model = tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(None, 142)),  # Input layer for variable-length sequences with 142 features
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
#     # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
#     # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
#     tf.keras.layers.Dense(64),  # Optional dense layer
#     tf.keras.layers.Dense(35, activation='relu')  # Output layer, size 35 to match your output data dimension
# ])

# # Compile the model
# optimizer = tf.keras.optimizers.Adam()
# model.compile(optimizer=optimizer, loss=custom_loss)

# # Assuming `dataset` is already prepared and `num_of_batches` is defined
# # Training the model

# dataset = dataset.repeat()
# with tf.device('/gpu:0'):
#     model.fit(dataset, epochs=2, steps_per_epoch=num_of_batches)
# # After running, you can monitor GPU usage using `nvidia-smi` from the command line


In [43]:
# from dotenv import load_dotenv
# import os
# #setting data path
# load_dotenv()
# project_path = os.getenv('PROJECT_PATH')
# model.save(f'{project_path}/model_name.h5')

In [ ]:
# input_data = tf.ragged.constant([processed_audio_list[0]])  # Example input with sequence length 100 and 142 features

# # Generate output using the loaded model
# output = model.predict(input_data)

In [ ]:
# for h in output[0]:
#     print(h)

In [ ]:
# from makeBeatmap import make_beatmap_from_output
# make_beatmap_from_output(df.iloc[0],output[0])